In [1]:
from PolishSystem.read_data import get_pRef_from_vectors, get_vectors_file_name, get_fitness_file_name

# this section will generate the search objectives
size = 20
method = "qmc"
fitness_column_to_use = 0
data_folder = r"C:\Users\gac8\PycharmProjects\PSSearch\data\retail_forecasting"

sessions = get_pRef_from_vectors(get_vectors_file_name(data_folder, size, method), 
                                     get_fitness_file_name(data_folder, size, method), fitness_column_to_use)


print(sessions)

PRef with 52626 samples, mean = 0.88


In [16]:


from Core.PS import PS
from SimplifiedSystem.ps_search_utils import get_metric_function
from Core.PRef import PRef
from PolishSystem.PolishPSDecisionTree import PolishPSDecisionTree
from SimplifiedSystem.PSSearchSettings import get_default_search_settings

search_settings = get_default_search_settings()
search_settings.verbose = True
search_settings.ps_search_budget=1000   # the recommended quantity is 5000 (this is per split)


def get_objectives_for_partition(pRef: PRef):
    
    # get_metric_function("sample_count", sessions) to count how many sessions are covered by a pattern
        
    # return [get_metric_function("simplicity"), get_metric_function("variance", pRef)] # PS-SW
    return [get_metric_function("variance", pRef), get_metric_function("sample_count", sessions)]


tree = PolishPSDecisionTree(maximum_depth=3, 
                            search_settings=search_settings,
                            get_objectives_for_partition=get_objectives_for_partition)

tree.train_from_pRef(sessions)

Splitting a pRef of size 52626
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       61 |      4 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |      161 |      5 |  0.000000E+00 |  0.000000E+00 |  0.0811652278 |         ideal
     3 |      261 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0189362538 |         nadir
     4 |      361 |      8 |  0.000000E+00 |  0.000000E+00 |  0.0776533845 |             f
     5 |      461 |      8 |  0.000000E+00 |  0.000000E+00 |  0.0089356848 |             f
     6 |      561 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0128825505 |             f
     7 |      661 |      9 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     8 |      761 |      9 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |      861 |      9 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |      961 |      9 |  0.000000E+00 |  0.000000E+00 

In [3]:
tree.print_ASCII()

Root
(Branch) 	
	[* * * * * 1 * * * * * * * 1 * * 1 * * *][0.034, -13360.000, ]	
	stats:n->52626, average->0.8756650999718619, variance->0.034203744445680306, sd->0.18494254363363857, mse->0.034203744445680306, mae->0.13565800144060147, min->0.0007308570784516633, max->1.0
├── Matching
│   (Branch) 	
│   	[* * * * * 1 * * * * * * * 1 * * 1 * * *][0.034, -13360.000, ]	
│   	stats:n->13360, average->0.8576769783881334, variance->0.04011050537270586, sd->0.20027607289116156, mse->0.04011050537270586, mae->0.1520256673240885, min->0.004234409425407648, max->1.0
│   ├── Matching
│   │   (Branch) 	
│   │   	[* * * * * 1 * * * * * * * 1 * * 1 * * *][0.034, -13360.000, ]	
│   │   	stats:n->13360, average->0.8576769783881334, variance->0.04011050537270586, sd->0.20027607289116156, mse->0.04011050537270586, mae->0.1520256673240885, min->0.004234409425407648, max->1.0
│   │   ├── Matching
│   │   │   (Leaf) stats:n->13360, average->0.8576769783881334, variance->0.04011050537270586, sd->0.20027607

In [15]:
import numpy as np
from DecisionTree.PSDecisionTree import PSDecisionTreeNode


def repr_95_confidence_interval(node: PSDecisionTreeNode):
    fitnesses = node.fitnesses
    
    if len(fitnesses) < 2:
        return f"{fitnesses}"
    mean = np.average(fitnesses)
    a, b = np.percentile(fitnesses, [2.5, 97.5])

    return f"(Branch), n = {len(fitnesses)}, {mean = :.2f},  confidence interval = [{a:.3f}, {b:.3f}]"

tree.print_ASCII(custom_partition_repr=repr_95_confidence_interval)
    

Root
(Branch) 	
	[* * 1 * * * * * * * * * * * * * * * * *][0.034, -8711.000, ]	
	(Branch), n = 52626, mean = 0.88,  confidence interval = [0.318, 1.000]
├── Matching
│   (Branch) 	
│   	[* * * * * * * * * * * * * * * * * * * *][0.034, -52626.000, ]	
│   	(Branch), n = 8711, mean = 0.85,  confidence interval = [0.257, 1.000]
│   ├── Matching
│   │   (Branch) 	
│   │   	[* * * * * * * * * * * * * * * * * * * *][0.034, -52626.000, ]	
│   │   	(Branch), n = 8711, mean = 0.85,  confidence interval = [0.257, 1.000]
│   │   ├── Matching
│   │   │   (Leaf) (Branch), n = 8711, mean = 0.85,  confidence interval = [0.257, 1.000]
│   │   └── NOT matching
│   │       (Leaf) []
│   └── NOT matching
│       (Leaf) []
└── NOT matching
    (Branch) 	
    	[* * * * * * * * * * * * * * * * * * * *][0.034, -52626.000, ]	
    	(Branch), n = 43915, mean = 0.88,  confidence interval = [0.337, 1.000]
    ├── Matching
    │   (Branch) 	
    │   	[* * * * * * * * * * * * * * * * * * * *][0.034, -52626.000, ]	
 